In [42]:
import numpy as np
import igl
import meshplot as mp
from scipy.spatial.transform import Rotation
import scipy
import ipywidgets as iw
import time


In [43]:
v, f = igl.read_triangle_mesh('data/woody-hi.off')
labels = np.load('data/woody-hi.label.npy').astype(int)
v -= v.min(axis=0)
v /= v.max()

In [44]:
handle_vertex_positions = v.copy()
pos_f_saver = np.zeros((labels.max() + 1, 6))
def pos_f(s,x,y,z, α, β, γ):
    slices = (labels==s)
    r = Rotation.from_euler('xyz', [α, β, γ], degrees=True)
    v_slice = v[slices] + np.array([[x,y,z]])
    center = v_slice.mean(axis=0)
    handle_vertex_positions[slices] = r.apply(v_slice - center) + center
    pos_f_saver[s - 1] = [x,y,z,α,β,γ]
    t0 = time.time()
    v_deformed = pos_f.deformer(handle_vertex_positions)
    p.update_object(vertices = v_deformed)
    t1 = time.time()
    print('FPS', 1/(t1 - t0))
pos_f.deformer = lambda x:x

In [45]:
def widgets_wrapper():
    segment_widget = iw.Dropdown(options=np.arange(labels.max()) + 1)
    translate_widget = {i:iw.FloatSlider(min=-1, max=1, value=0) 
                        for i in 'xyz'}
    rotate_widget = {a:iw.FloatSlider(min=-90, max=90, value=0, step=1) 
                     for a in 'αβγ'}

    def update_seg(*args):
        (translate_widget['x'].value,translate_widget['y'].value,
        translate_widget['z'].value,
        rotate_widget['α'].value,rotate_widget['β'].value,
        rotate_widget['γ'].value) = pos_f_saver[segment_widget.value]
    segment_widget.observe(update_seg, 'value')
    widgets_dict = dict(s=segment_widget)
    widgets_dict.update(translate_widget)
    widgets_dict.update(rotate_widget)
    return widgets_dict

In [46]:
def position_deformer(target_pos):
    '''Fill in this function to change positions'''
    return target_pos
''' (Optional) Register this function to perform interactive deformation
pos_f.deformer = position_deformer
'''

' (Optional) Register this function to perform interactive deformation\npos_f.deformer = position_deformer\n'

In [47]:
# part 1 & 2(removing complex features)
def remove_high_frequency_features(v, f, labels):
    """
    This function removes high frequency details from the mesh
    """
    
    lw = igl.cotmatrix(v, f)
    m = igl.massmatrix(v, f, igl.MASSMATRIX_TYPE_BARYCENTRIC)
    m_inv = scipy.sparse.diags(1/(m.diagonal()))
        
    A = lw@m_inv@lw
    # computing number of free variables
    fi = (labels == 0)
    ci = (labels != 0)
    nf = len(v[fi])
    A_ff = A[:,fi][fi]
    A_fc = A[:,ci][fi]
    v_c = v[ci]
        
    v_solve = scipy.sparse.linalg.spsolve(A_ff, -A_fc@v_c)
    v_smooth = np.zeros_like(v)
    v_smooth[fi] = v_solve
    v_smooth[ci] = v[ci]
    
    return v_smooth, A_ff, A_fc

v_smooth, A_ff, A_fc = remove_high_frequency_features(v, f, labels)    
p = mp.plot(v_smooth, f, c = labels)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.4313118…

In [48]:
# part 3
def compute_local_displacements(v, vs, f):
    """
    This function computes the displacements of the free vertices after smoothing in the local
    frames
    Input:
        v : original location of vertices
        vs : smooth mesh vertices without high frequency features
        d_vs : deformed mesh vertices
        f : faces array
    """

    
    d_tmp = v - vs
    arr_l_e = np.zeros_like(v[:,0], dtype=int)
    d = np.zeros_like(d_tmp)
    
    AA = igl.adjacency_list(f)
    N = igl.per_vertex_normals(vs,f)

    for i in range(len(v)):
        # projecting neighbours into tangent plane
        p_ei = v[AA[i]]-((v[AA[i]] - v[i]).dot(N[i]))[:,None]*N[i]
        # computing the neighbour with the largest norm
        p_ei -= v[i]
        norm_pe_i = np.linalg.norm(p_ei, axis = 1)
        l_e = np.argmax(norm_pe_i)
        # storing longest edge
        arr_l_e[i] = int(AA[i][l_e])
        # normalizing the edge
        p_ei = p_ei[l_e]/norm_pe_i[l_e]
        # computing third basis
        k_ei = np.cross(p_ei,N[i])
        # computing displacement in local frame
        d[i] = np.array([d_tmp[i].dot(p_ei),d_tmp[i].dot(k_ei), d_tmp[i].dot(N[i])]) 
    
    return d, arr_l_e
    
d, arr_l_e = compute_local_displacements(v, v_smooth, f)

In [49]:
## Widget UI

p = mp.plot(handle_vertex_positions, f, c=labels)
iw.interact(pos_f,
            **widgets_wrapper())

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.4306930…

interactive(children=(Dropdown(description='s', options=(1, 2, 3), value=1), FloatSlider(value=0.0, descriptio…

<function __main__.pos_f(s, x, y, z, α, β, γ)>

In [57]:
import time
def project_features(v_deformed, f, d, arr_l_e, labels, A, b, no_hf_features = False):
    """
    This function projects the features back to the deformed mesh
    Input:
        v_deformed : deformed mesh
        f : faces
        d : local frame displacements
        arr_l_e : the vertex with longest edge
        labels : constraints
        A : A matrix to solve linear system
        b : matrix rhs
        no_hf_features : trigger to show smooth mesh after deformation
    """
    vd = v_deformed.copy()
    vd[labels == 0] = scipy.sparse.linalg.spsolve(A_ff, -A_fc@vd[labels != 0])
    et = time.time()
    if not no_hf_features:
        N = igl.per_vertex_normals(vd,f)
        for i in range(len(arr_l_e)):
            p_ei = v[arr_l_e[i]] - v[i] - (v[arr_l_e[i]] - v[i]).dot(N[i])*N[i]
            p_ei /= np.linalg.norm(p_ei)
            k_ei = np.cross(p_ei, N[i])
            vd[i] += d[i][0]*p_ei + d[i][1]*k_ei + d[i][2]*N[i]

    return vd
    
# set to True to show the reconstructed deformed mesh with only smooth features
output = project_features(handle_vertex_positions, f, d, arr_l_e, labels, A_ff, A_fc, False)

0.018065214157104492
0.1185910701751709


In [55]:
p = mp.plot(output, f, c=labels)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.4306930…